In [1]:
from os.path import expanduser
import pandas as pd
import simplejson
import numpy as np
import urllib
import json

In [2]:
# get the google api key
home = expanduser("~")
with open(home + '/.googleapi') as infile:
    apikey = infile.next().strip()

In [3]:
# read in the metadata file
meta_hdf_path = '/Users/bohr/workspace/capstone/DSE_Cohort2_Traffic_Capstone/cohort2/data/meta_2015.hdf'
meta_frame = pd.read_hdf(meta_hdf_path, 'meta_2015')
meta_frame.head()

,ID,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,Length,Type,Lanes,Name,User_ID_1,User_ID_2,User_ID_3,User_ID_4,file_date
584,1113072,56,W,11,73,66000.0,7.383,7.885,32.955202,-117.124689,0.452,ML,2,Black Mountain Rd,314,NaN,NaN,NaN,2015_01_01
585,1113073,56,W,11,73,66000.0,7.383,7.885,32.955202,-117.124689,NaN,OR,3,BLACK MOUNTAIN RD,314,NaN,NaN,NaN,2015_01_01
665,1113680,56,E,11,73,66000.0,6.862,7.364,32.953394,-117.133404,0.999,ML,3,BLACK MOUNTAIN RD,434,NaN,NaN,NaN,2015_01_01
666,1113683,56,W,11,73,66000.0,7.383,7.885,32.955202,-117.124689,NaN,FR,2,BLK MOUNTAIN - WB 56,314,NaN,NaN,NaN,2015_01_01
1034,1119041,125,N,11,73,NaN,1.433,3.282,32.608637,-116.967247,0.792,ML,3,1 MI S/O BIRCH RD,19106,NaN,NaN,NaN,2015_01_01


In [4]:
meta_frame.count()

ID           1541
Fwy          1541
Dir          1541
District     1541
County       1541
City         1318
State_PM     1541
Abs_PM       1541
Latitude     1539
Longitude    1539
Length        932
Type         1541
Lanes        1541
Name         1541
User_ID_1    1541
User_ID_2       0
User_ID_3       0
User_ID_4       0
file_date    1541
dtype: int64

In [6]:
meta_frame.Type.unique()

array(['ML', 'OR', 'FR', 'FF', 'HV', 'CH', 'CD'], dtype=object)

In [5]:
meta_frame[meta_frame.Latitude.isnull()]

,ID,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,Length,Type,Lanes,Name,User_ID_1,User_ID_2,User_ID_3,User_ID_4,file_date
767,1114649,805,S,11,73,66000.0,28.811,28.662,NaN,NaN,0.651,ML,4,S/B AT JCT I-5,4045,NaN,NaN,NaN,2015_12_17
1324,1125383,52,W,11,73,70224.0,14.756,14.756,NaN,NaN,0.695,ML,2,52 WB from 125 Conn,43812,NaN,NaN,NaN,2015_12_17


In [6]:
meta_frame_no_nan = meta_frame.dropna(axis='index', subset=['Latitude', 'Longitude'])
meta_frame_no_nan.index = np.arange(0, len(meta_frame_no_nan))
meta_frame_no_nan['lat_lon'] = meta_frame_no_nan['Latitude'].map(str) + ',' + meta_frame_no_nan['Longitude'].map(str)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [7]:
# spot check
meta_frame_no_nan[meta_frame_no_nan.ID == 1114649]

,ID,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,Length,Type,Lanes,Name,User_ID_1,User_ID_2,User_ID_3,User_ID_4,file_date,lat_lon


In [8]:
meta_frame_no_nan.count()

ID           1539
Fwy          1539
Dir          1539
District     1539
County       1539
City         1316
State_PM     1539
Abs_PM       1539
Latitude     1539
Longitude    1539
Length        930
Type         1539
Lanes        1539
Name         1539
User_ID_1    1539
User_ID_2       0
User_ID_3       0
User_ID_4       0
file_date    1539
lat_lon      1539
dtype: int64

In [9]:
ELEVATION_BASE_URL = 'https://maps.googleapis.com/maps/api/elevation/json'
def getElevation(lat_lon_list=None, **elvtn_args):
    """
    This function will get the elevation from the google api
    """
    result = None
    elvtn_args.update({
        'locations': "|".join(lat_lon_list),
        'key': apikey
    })

    url = ELEVATION_BASE_URL + '?' + urllib.urlencode(elvtn_args)
    response = simplejson.load(urllib.urlopen(url))

    if 'status' not in response or response['status'] != 'OK':
        print "bad response"
    else:
        result = [item['elevation'] for item in response['results']]
    return result

In [236]:
# # chunk the data into parts of 50
# id_master = []
# elevation_master = []

# chunk = 50
# for item in range(0, 31):
#     start = item*chunk
#     end = (item+1)*chunk
    
#     id_master += list(meta_frame_no_nan['ID'][start:end].values)
#     elevation_master += getElevation(list(meta_frame_no_nan['lat_lon'][start:end]))
    
# print len(id_master)
# print len(elevation_master)

1539
1539


In [17]:
# elev_df = pd.DataFrame({'ID': id_master, 'elevation': elevation_master})

In [10]:
meta_frame.columns

Index([u'ID', u'Fwy', u'Dir', u'District', u'County', u'City', u'State_PM',
       u'Abs_PM', u'Latitude', u'Longitude', u'Length', u'Type', u'Lanes',
       u'Name', u'User_ID_1', u'User_ID_2', u'User_ID_3', u'User_ID_4',
       u'file_date'],
      dtype='object')

In [20]:
# elev_df.columns

In [21]:
# new_meta_frame = pd.merge(meta_frame, elev_df, how='left')

In [11]:
# find bad data...
# new_meta_frame[new_meta_frame.elevation.isnull()]

In [12]:
# Fix bad data via manual lookup
# new_meta_frame.loc[884, 'Latitude'] = 32.966531
# new_meta_frame.loc[884, 'Longitude'] = -117.2255
# new_meta_frame.iloc[884]

In [13]:
# new_meta_frame.loc[1441, 'Latitude'] = 32.836534
# new_meta_frame.loc[1441, 'Longitude'] = -117.00755
# new_meta_frame.iloc[1441]

In [14]:
# result_884 = getElevation(['32.9665,-117.2255'])
# result_1441 = getElevation(['32.8365,-117.00755'])

In [15]:
# new_meta_frame.loc[884, 'elevation'] = result_884[0]
# new_meta_frame.loc[1441, 'elevation'] = result_1441[0]

In [16]:
# Note: all elevations are in meters.
# new_meta_frame.to_csv('../data/meta_2015_with_elev.csv')

# Calculate slope between stations

In [8]:
new_meta_frame = pd.read_csv('../data/meta_2015_with_elev.csv')

In [9]:
new_meta_frame.columns

Index([u'Unnamed: 0', u'Unnamed: 0.1', u'Unnamed: 0.1.1', u'ID', u'Fwy',
       u'Dir', u'District', u'County', u'City', u'State_PM', u'Abs_PM',
       u'Latitude', u'Longitude', u'Length', u'Type', u'Lanes', u'Name',
       u'User_ID_1', u'User_ID_2', u'User_ID_3', u'User_ID_4', u'file_date',
       u'elevation'],
      dtype='object')

In [10]:
new_meta_frame.Type.unique()

array(['ML', 'OR', 'FR', 'FF', 'HV', 'CH', 'CD'], dtype=object)

In [6]:
# new_meta_frame[new_meta_frame.Type == 'ML'].ID.count()

In [17]:
def create_freeway_vectors(frame_to_use, columns_to_select=['ID', 'Latitude', 'Longitude', 'Abs_PM', 'Lanes']):
    to_loop = new_meta_frame.groupby(['Fwy', 'Dir'])['ID'].count().reset_index()[['Fwy', 'Dir']].values
    
    ret = {}
    for Fwy, Dir in to_loop:
        if Dir == "N":
            sort_order = ('Abs_PM', True)        
        elif Dir == "S":
            sort_order = ('Abs_PM', True)        
        elif Dir == "E":
            sort_order = ('Abs_PM', True)        
        elif Dir == "W":
            sort_order = ('Abs_PM', True)
            
        tmp = frame_to_use[(frame_to_use.Fwy == Fwy) & (frame_to_use.Dir == Dir)
                                                  & (frame_to_use.Type == 'ML')]\
            .sort_values(by=sort_order[0], ascending=sort_order[1])[columns_to_select].drop_duplicates()
        tmp['order'] = pd.Series(index=tmp.index, data=sorted(range(0, len(tmp.ID)), reverse=(not sort_order[1])))
        ret["%s_%s" % (Fwy, Dir)] = tmp
    return ret 

In [18]:
freeway_vectors = create_freeway_vectors(new_meta_frame)

In [19]:
len(freeway_vectors)

26

In [20]:
freeway_vectors

{'125_N':           ID   Latitude   Longitude  Abs_PM  Lanes  order
 18   1119021  32.595673 -116.964628   2.344      2      0
 4    1119041  32.608637 -116.967247   3.282      3      1
 19   1119050  32.617282 -116.971114   3.928      3      2
 6    1119059  32.619301 -116.971171   4.067      2      3
 23   1119075  32.623484 -116.971141   4.356      2      4
 25   1119085  32.625784 -116.971135   4.515      3      5
 9    1119094  32.629142 -116.971145   4.747      2      6
 28   1119102  32.634145 -116.971152   5.092      2      7
 11   1119110  32.641088 -116.970426   5.575      2      8
 32   1119126  32.649208 -116.970687   6.136      2      9
 34   1119135  32.656460 -116.972951   6.657      2     10
 36   1119144  32.658804 -116.975168   6.865      2     11
 38   1119162  32.666777 -116.984009   7.634      2     12
 40   1119171  32.671414 -116.984862   7.958      2     13
 30   1119118  32.675550 -116.985470   8.246      2     14
 45   1119205  32.684722 -117.009803  10.142   

In [10]:
print freeway_vectors['15_N'].head()

           ID   Latitude   Longitude  Abs_PM  order
1138  1119689  32.701162 -117.120639   0.734      0
1139  1119694  32.705882 -117.120433   1.066      1
84    1122948  32.715149 -117.117717   1.713      2
82    1122942  32.716837 -117.117751   1.829      3
1105  1118886  32.723255 -117.115308   2.302      4


In [23]:
ELEVATION_BASE_URL = 'https://maps.googleapis.com/maps/api/elevation/json'
def getElevationPath(lat_lon_list=None, samples=10):
    """
    This function will get the elevation from the google api
    """
    result = None
    elvtn_args = {
        'path': "|".join(lat_lon_list),
        'samples': samples,
        'key': apikey
    }

    url = ELEVATION_BASE_URL + '?' + urllib.urlencode(elvtn_args)
    response = simplejson.load(urllib.urlopen(url))

    if 'status' not in response or response['status'] != 'OK':
        print "bad response"
        print "response: %s" % response
    else:
        result = [item['elevation'] for item in response['results']]
    return result

In [34]:
# Test the function
# result = getElevationPath(['32.834748,-117.003404', '32.809436,-117.005295'], samples=18)

In [24]:
# loop through all freeways and get the results
# calculations will be done afterwards to debug while not pulling from google's api
# raw_results = {}
# counter = 0
# for key in freeway_vectors:
#     print "on freeway: %s" % key
#     if counter % 3:
#         print "counter: %s" % counter
#     frame = freeway_vectors[key]
#     count = frame.ID.count()
#     for index in range(1, count):
#         distance = abs(frame.iloc[index-1].Abs_PM - frame.iloc[index].Abs_PM)

#         # get enough samples for .1 resolution
#         samples = int(round(distance, 1)*10)
#         lat1, lon1 = frame.iloc[index-1][['Latitude', 'Longitude']]
#         lat_lon1 = '%s,%s' % (lat1, lon1)
#         lat2, lon2 = frame.iloc[index][['Latitude', 'Longitude']]
#         lat_lon2 = '%s,%s' % (lat2, lon2)
#         if samples < 2:
#             samples = 2
#         result = getElevationPath([lat_lon1, lat_lon2], samples=samples)
        
#         start_id = frame.iloc[index-1].ID
#         end_id = frame.iloc[index].ID
#         if key not in raw_results:
#             raw_results[key] = {}
#         raw_results[key]['%s_%s' % (start_id, end_id)] = result
#     counter += 1

In [36]:
# calculate the grade for each station
# id_list = []
# grade_list = []
# for key in freeway_vectors:
#     frame = freeway_vectors[key]
#     count = frame.ID.count()
#     for index in range(1, count+1):
#         # absolute value of run in case we walk the freeway backwards
#         if index < count:
#             run = abs(frame.iloc[index-1].Abs_PM - frame.iloc[index].Abs_PM)
#         else:
#             run = abs(frame.iloc[index-2].Abs_PM - frame.iloc[index-1].Abs_PM)

#         # sometimes there are multiple stations on the same mile marker...
#         if run == 0:
#             if index < count:
#                 pad = 0
#             else:
#                 pad = 1
#             for i in range(1, 5):
#                 run = abs(frame.iloc[index-i].Abs_PM - frame.iloc[index-pad].Abs_PM)
#                 if run != 0:
#                     break
# #         print "run: %s" % run

#         if index < count:
#             start_id = frame.iloc[index-1].ID
#             end_id = frame.iloc[index].ID
#         else:
#             start_id = frame.iloc[index-2].ID
#             end_id = frame.iloc[index-1].ID
            
#         raw_elevations = raw_results[key]['%s_%s' % (start_id, end_id)]
# #         print "raw_elevations: %s" % raw_elevations
        
#         run_between_points = run/len(raw_elevations)
# #         print "run_between_points: %s" % run_between_points

#         # use the closest point to calculate the grade
#         if index == count:
#             # if at the end of the vector, then use the last raw_elevations
#             grade = np.arctan((raw_elevations[-1] - raw_elevations[-2])/float(run_between_points))
#         else:
#             grade = np.arctan((raw_elevations[1] - raw_elevations[0])/float(run_between_points))
        
#         id_list.append(start_id)
#         grade_list.append(grade)

In [39]:
# grade_df = pd.DataFrame({'ID': id_list, 'grade': grade_list})

In [40]:
# grade_df[grade_df.grade.isnull()]

,ID,grade


In [41]:
# new_meta_frame_with_grade = pd.merge(new_meta_frame, grade_df, how='inner', on='ID')

In [46]:
# new_meta_frame_with_grade_ML[[u'ID', u'Fwy', u'Dir', u'Abs_PM', u'Latitude', u'Longitude', u'Lanes', u'Name',
#      u'elevation', u'grade']].to_csv('../data/meta_2015_with_elev_and_grade.csv')

In [45]:
# new_meta_frame_with_grade_ML[[u'ID', u'Fwy', u'Dir', u'Abs_PM', u'Latitude', u'Longitude', u'Lanes', u'Name',
#      u'elevation', u'grade']].to_json('../data/meta_ML_2015_with_elev_and_grade_mod.json')

In [12]:
new_meta_frame_with_grade_ML = pd.read_csv('../data/meta_2015_with_elev_and_grade.csv')
new_meta_frame_with_grade_ML['Type'] = 'ML'
new_meta_frame_with_grade_ML.head()

,Unnamed: 0,ID,Fwy,Dir,Abs_PM,Latitude,Longitude,Lanes,Name,elevation,grade,Type
0,0,1113072,56,W,7.885,32.955202,-117.124689,2,Black Mountain Rd,178.243729,-1.559401,ML
1,1,1113680,56,E,7.364,32.953394,-117.133404,3,BLACK MOUNTAIN RD,150.936234,1.545684,ML
2,2,1119041,125,N,3.282,32.608637,-116.967247,3,1 MI S/O BIRCH RD,163.306900,1.563082,ML
3,3,1119042,125,S,5.428,32.637029,-116.971446,2,1 MI S/O BIRCH RD,169.974411,-1.532295,ML
4,4,1119059,125,N,4.067,32.619301,-116.971171,2,SOUTH SIDE BIRCH RD,180.948593,1.520973,ML


In [13]:
new_meta_frame_with_grade_ML.count()

Unnamed: 0    839
ID            839
Fwy           839
Dir           839
Abs_PM        839
Latitude      839
Longitude     839
Lanes         839
Name          839
elevation     839
grade         839
Type          839
dtype: int64

In [35]:
freeway_vectors_update = create_freeway_vectors(
    new_meta_frame_with_grade_ML, [u'ID', u'Fwy', u'Dir', u'Abs_PM', u'Latitude', u'Longitude', u'Lanes', u'Name',
     u'elevation'])

In [36]:
freeway_vectors_update['805_S']

,ID,Fwy,Dir,Abs_PM,Latitude,Longitude,Lanes,Name,elevation,order
408,1114649,805,S,28.662,32.966531,-117.225500,4,S/B AT JCT I-5,50.037918,59
828,1125913,805,S,27.483,32.894648,-117.214511,2,805 SB Bypass LN,34.468193,58
825,1125897,805,S,27.482,32.894640,-117.214497,2,805 SB Bypass N/O,34.874592,57
775,1123089,805,S,27.478,32.894604,-117.214442,4,805 SB N/O Sorrento,35.012482,56
753,1122678,805,S,27.478,32.894604,-117.214442,4,805 SB Sorrento,35.012482,55
811,1125769,805,S,26.986,32.891018,-117.207135,4,Mira Mesa Bl,48.135792,54
358,1113473,805,S,26.801,32.889096,-117.204916,5,Sorrento Valley Rd,59.175648,53
502,1116049,805,S,26.251,32.881653,-117.201619,5,N/O MIRAMAR RD,92.293709,52
144,1108413,805,S,25.790,32.875201,-117.199467,4,S805 LAJOLLA/MIRAMAR,100.892960,51
289,1108764,805,S,25.100,32.867100,-117.192516,4,Nobel Dr,104.759689,50


In [37]:
freeway_vectors_update['5_S']

,ID,Fwy,Dir,Abs_PM,Latitude,Longitude,Lanes,Name,elevation,order
82,1121740,5,S,72.008,33.394031,-117.592254,5,5 SB Cristianitos Rd,25.006939,101
709,1121734,5,S,70.878,33.383349,-117.578043,4,5 SB Basilone Rd,15.903173,100
708,1121728,5,S,68.279,33.362845,-117.540736,4,5 SB Nuclear Plant,53.328392,99
705,1121722,5,S,65.449,33.336536,-117.503249,4,5 SB Truck Scales,52.426800,98
704,1121715,5,S,65.448,33.336526,-117.503236,4,5 NB Track Scale,52.389683,97
703,1121712,5,S,62.419,33.303834,-117.470867,4,5 SB Las Pulgas Rd,24.570593,96
701,1121702,5,S,60.429,33.281440,-117.449446,4,5 SB N/O Rest Area,24.727163,95
663,1119997,5,S,58.329,33.257953,-117.426547,4,1M S/O ALISO CREEK,22.016130,94
660,1119978,5,S,56.858,33.240152,-117.413376,4,.5 N/O STA MARGRITA,17.393425,93
658,1119966,5,S,55.858,33.230196,-117.400904,4,S/O SANTA MARGARITA,5.098459,92


In [27]:
def sorted_func(x):
    values = x.split('_')
    Fwy = int(values[0])
    Dir = values[1]
    if Dir == 'N' or Dir == 'E':
        dir_weight = 0
    else:
        dir_weight = 1
    return Fwy + dir_weight

In [38]:
sorted(freeway_vectors_update.keys(), key=sorted_func)

['5_N',
 '5_S',
 '8_E',
 '8_W',
 '15_N',
 '15_S',
 '52_E',
 '52_W',
 '54_E',
 '54_W',
 '56_E',
 '56_W',
 '78_E',
 '78_W',
 '94_E',
 '94_W',
 '125_N',
 '125_S',
 '163_N',
 '163_S',
 '805_N',
 '805_S',
 '905_E',
 '905_W']

In [151]:
for freeway_key in freeway_vectors_update:
    data = freeway_vectors_update[freeway_key][[u'ID', u'Fwy', u'Dir', u'Abs_PM', u'Latitude', u'Longitude', 
                                                u'Lanes', u'Name', u'elevation', 'order']]
#     data['order'] = pd.Series(index=data.index, data=range(1, len(data)+1))
    data[[u'ID', u'Fwy', u'Dir', u'Abs_PM', u'Latitude', u'Longitude', u'Lanes', u'Name',
          u'elevation', 'order']].to_json('../data/%s_ML_2015.json' % freeway_key, orient='records')

In [14]:
!pwd

/Users/bohr/workspace/capstone/DSE_Cohort2_Traffic_Capstone/cohort2/exploration


In [ ]:
files=!ls ../data/*_ML_2015.json

In [15]:
files

['../data/125_N_ML_2015.json',
 '../data/125_S_ML_2015.json',
 '../data/15_N_ML_2015.json',
 '../data/15_S_ML_2015.json',
 '../data/163_N_ML_2015.json',
 '../data/163_S_ML_2015.json',
 '../data/52_E_ML_2015.json',
 '../data/52_W_ML_2015.json',
 '../data/54_E_ML_2015.json',
 '../data/54_W_ML_2015.json',
 '../data/56_E_ML_2015.json',
 '../data/56_W_ML_2015.json',
 '../data/5_N_ML_2015.json',
 '../data/5_S_ML_2015.json',
 '../data/67_N_ML_2015.json',
 '../data/67_S_ML_2015.json',
 '../data/78_E_ML_2015.json',
 '../data/78_W_ML_2015.json',
 '../data/805_N_ML_2015.json',
 '../data/805_S_ML_2015.json',
 '../data/8_E_ML_2015.json',
 '../data/8_W_ML_2015.json',
 '../data/905_E_ML_2015.json',
 '../data/905_W_ML_2015.json',
 '../data/94_E_ML_2015.json',
 '../data/94_W_ML_2015.json']

In [154]:
# test_final = json.load(open('../data/2015_ML_d11_geojson_points.json'))

In [155]:
# len(test_final['125_S']['data']['features'])

36

In [31]:
# final = {}

# for filename in files:
#     new_geojson = {'type': 'FeatureCollection', 'features': []}
#     print "geojson len: %s" % len(new_geojson['features'])
#     data = json.load(open(filename))
#     key = '_'.join(filename.split('/')[2].split('_')[0:2])
#     print key
#     if key == '15_N':
#         visible = False
#     else:
#         visible = False

#     print len(data)
#     for row in data:
#         properties = {'key': key,
#                       'ID': row['ID'],
#                       'Lanes': row['Lanes'],
#                       'Name': row['Name'],
#                       'Abs_PM': row['Abs_PM'],
#                       'Elevation': row['elevation'],
#                       'Order': row['order']}
#         geometry = {'type': "Point", "coordinates": [row['Longitude'], row['Latitude']]}
#         temp = {'type': 'Feature', 'properties': properties, "geometry": geometry}
#         new_geojson['features'].append(temp)

#     print "geojson len: %s" % len(new_geojson['features'])
#     final[key] = {'visible': visible, 'data': new_geojson}
# json.dump(final, open('../data/2015_ML_d11_geojson_points.json', 'w'))

In [77]:
final = {}
for filename in files:
    data = json.load(open(filename))
    key = '_'.join(filename.split('/')[2].split('_')[0:2])
    print key
    if key == '15_N':
        visible = True
    else:
        visible = False
    final[key] = {'visible': visible, 'data': data}
json.dump(final, open('../data/2015_ML_d11.json', 'w'))

[{u'Lanes': 2, u'Name': u'OTAY RIVER BRIDGE NE', u'Longitude': -116.964628, u'Fwy': 125, u'Latitude': 32.595673, u'elevation': 95.2415084839, u'Abs_PM': 2.344, u'Dir': u'N'}, {u'Lanes': 3, u'Name': u'1 MI S/O BIRCH RD', u'Longitude': -116.967247, u'Fwy': 125, u'Latitude': 32.608637, u'elevation': 163.306900024, u'Abs_PM': 3.282, u'Dir': u'N'}, {u'Lanes': 3, u'Name': u'125 SB S/O BIRCH RD', u'Longitude': -116.971114, u'Fwy': 125, u'Latitude': 32.617282, u'elevation': 178.962783813, u'Abs_PM': 3.928, u'Dir': u'N'}, {u'Lanes': 2, u'Name': u'SOUTH SIDE BIRCH RD', u'Longitude': -116.971171, u'Fwy': 125, u'Latitude': 32.619301, u'elevation': 180.94859314, u'Abs_PM': 4.067, u'Dir': u'N'}, {u'Lanes': 2, u'Name': u'125 SB N/O BIRCH RD', u'Longitude': -116.971141, u'Fwy': 125, u'Latitude': 32.623484, u'elevation': 184.006469727, u'Abs_PM': 4.356, u'Dir': u'N'}, {u'Lanes': 3, u'Name': u'.5MI S/O OLYMPIC PKW', u'Longitude': -116.971135, u'Fwy': 125, u'Latitude': 32.625784, u'elevation': 183.279968

In [72]:
new_meta_frame_with_grade_ML[[u'ID', u'Fwy', u'Dir', u'Abs_PM', u'Latitude', u'Longitude', u'Lanes', u'Name',
     u'elevation', u'grade']].elevation.describe()

count    839.000000
mean      90.269059
std       68.351223
min        3.167338
25%       25.705447
50%       77.214447
75%      142.640480
max      332.536011
Name: elevation, dtype: float64

In [48]:
# loop through all freeways and get the results
# calculations will be done afterwards to debug while not pulling from google's api
raw_results = {}
counter = 0
to_review = {}
for key in freeway_vectors_update:
    print "on freeway: %s" % key
    frame = freeway_vectors_update[key]
    count = frame.ID.count()
    for index in range(1, count):
        distance = abs(frame.iloc[index-1].Abs_PM - frame.iloc[index].Abs_PM)

        if distance < .2:
            if key not in to_review:
                to_review[key] = []
            to_review[key].append(frame.iloc[index-1])
    counter += 1

on freeway: 125_S
on freeway: 905_E
on freeway: 54_W
on freeway: 125_N
on freeway: 52_W
on freeway: 67_N
on freeway: 56_E
on freeway: 67_S
on freeway: 56_W
on freeway: 905_W
on freeway: 94_E
on freeway: 52_E
on freeway: 78_W
on freeway: 5_N
on freeway: 15_N
on freeway: 15_S
on freeway: 94_W
on freeway: 78_E
on freeway: 54_E
on freeway: 5_S
on freeway: 8_E
on freeway: 163_N
on freeway: 805_S
on freeway: 8_W
on freeway: 805_N
on freeway: 163_S


In [49]:
to_review['5_N'][0:5]

[ID               1114734
 Fwy                    5
 Dir                    N
 Abs_PM              5.23
 Latitude         32.5938
 Longitude       -117.088
 Lanes                  5
 Name         S/O MAIN ST
 elevation        6.59395
 order                  8
 Name: 418, dtype: object, ID                       1123135
 Fwy                            5
 Dir                            N
 Abs_PM                     5.324
 Latitude                 32.5951
 Longitude               -117.089
 Lanes                          1
 Name         5 NB off to Main St
 elevation                7.44271
 order                          9
 Name: 780, dtype: object, ID                       1123331
 Fwy                            5
 Dir                            N
 Abs_PM                    10.713
 Latitude                 32.6705
 Longitude               -117.111
 Lanes                          1
 Name         Civic Cnt Dr to 5NB
 elevation                3.39357
 order                         19
 Name: 7

In [56]:
new_meta_frame_with_grade_ML[(new_meta_frame_with_grade_ML.Dir == 'N') & (new_meta_frame_with_grade_ML.Fwy == 5) &
                             (new_meta_frame_with_grade_ML.Lanes < 4)]

,Unnamed: 0,ID,Fwy,Dir,Abs_PM,Latitude,Longitude,Lanes,Name,elevation,grade,Type
434,434,1115277,5,N,32.795,32.932271,-117.241036,2,NB 5 BYP @ CRML VLLY,10.491953,1.553021,ML
771,771,1123081,5,N,30.532,32.903782,-117.223779,2,5 N By 5/805 Merge,20.922043,-1.546095,ML
780,780,1123135,5,N,5.324,32.595086,-117.088638,1,5 NB off to Main St,7.442711,1.514650,ML
790,790,1123331,5,N,10.713,32.670525,-117.111203,1,Civic Cnt Dr to 5NB,3.393571,1.566901,ML


In [62]:
new_meta_frame_with_grade_ML[(new_meta_frame_with_grade_ML.Dir == 'N') & (new_meta_frame_with_grade_ML.Fwy == 5)]\
    .ID.count()

109

In [58]:
2/float(109)

0.01834862385321101

<img src='screenshots/1123134.png'>
1123134 is really close to it's neighbor 1123135 0.001 miles

In [61]:
new_meta_frame_with_grade_ML[(new_meta_frame_with_grade_ML.Dir == 'S') & (new_meta_frame_with_grade_ML.Fwy == 5) &
                             (new_meta_frame_with_grade_ML.Lanes < 4)]

,Unnamed: 0,ID,Fwy,Dir,Abs_PM,Latitude,Longitude,Lanes,Name,elevation,grade,Type
410,410,1114660,5,S,30.812,32.907588,-117.226265,3,S/B AT JCT I-805,21.196461,-1.556265,ML
673,673,1120356,5,S,6.627,32.613849,-117.091173,3,SB 5 S/O L St,9.153790,1.436106,ML
769,769,1123072,5,S,30.532,32.903777,-117.224665,2,5 S Bypass S/O 5/805,17.328718,-1.562271,ML
779,779,1123134,5,S,5.324,32.595070,-117.088901,1,Main St to 5 SB,7.251188,1.268062,ML


<img src='screenshots/1116433.png'>
station 1116433 is really close to a neighboring station and is only 1 lane. seems like an incorrect station

In [75]:
new_meta_frame_with_grade_ML[(new_meta_frame_with_grade_ML.Dir == 'W') & (new_meta_frame_with_grade_ML.Fwy == 78) &
                             (new_meta_frame_with_grade_ML.Lanes < 2)]

,Unnamed: 0,ID,Fwy,Dir,Abs_PM,Latitude,Longitude,Lanes,Name,elevation,grade,Type
522,522,1116433,78,W,11.094,33.145066,-117.19219,1,Las Posas Rd,168.368027,1.507798,ML


In [ ]:
# 8, 52, 54, 56, 78E, 94, 905 has no issues
# 125, 163, 805 has no issues

In [91]:
new_meta_frame_with_grade_ML[(new_meta_frame_with_grade_ML.Dir == 'S') & (new_meta_frame_with_grade_ML.Fwy == 5) &
                             (new_meta_frame_with_grade_ML.Lanes < 3)]

,Unnamed: 0,ID,Fwy,Dir,Abs_PM,Latitude,Longitude,Lanes,Name,elevation,grade,Type
769,769,1123072,5,S,30.532,32.903777,-117.224665,2,5 S Bypass S/O 5/805,17.328718,-1.562271,ML
779,779,1123134,5,S,5.324,32.595070,-117.088901,1,Main St to 5 SB,7.251188,1.268062,ML


In [39]:
new_meta_frame_with_grade_ML[(new_meta_frame_with_grade_ML.Dir == 'S') & (new_meta_frame_with_grade_ML.Fwy == 15) &
                             (new_meta_frame_with_grade_ML.Lanes < 3)]

,Unnamed: 0,ID,Fwy,Dir,Abs_PM,Latitude,Longitude,Lanes,Name,elevation,grade,Type
63,63,1117920,15,S,1.703,32.715149,-117.117869,2,SB 15 @ 94,19.198471,1.547851,ML
67,67,1122945,15,S,1.703,32.715149,-117.117869,2,15 SB From 94,19.198471,0.000000,ML


In [40]:
new_meta_frame_with_grade_ML[(new_meta_frame_with_grade_ML.Dir == 'N') & (new_meta_frame_with_grade_ML.Fwy == 15) &
                             (new_meta_frame_with_grade_ML.Lanes < 3)]

,Unnamed: 0,ID,Fwy,Dir,Abs_PM,Latitude,Longitude,Lanes,Name,elevation,grade,Type
68,68,1122948,15,N,1.713,32.715149,-117.117717,2,94 EB From 15 NB,19.331848,1.522833,ML
735,735,1122502,15,N,30.470,33.109785,-117.095526,2,15 NB HOV N/O,218.190216,-1.550586,ML
783,783,1123179,15,N,3.059,32.733487,-117.111847,2,15NB From 805NB Conn,61.661972,0.000000,ML


<img src='screenshots/1108771.png'>
<img src='screenshots/1122502.png'>
station 1108771 is super close to 1122502 and 1122502 has a name of HOV plus is only two lanes as opposed to 5